# <div align="center" style="color: #ff5733;">New KPI Report</div>

## *Requestor is Jennyfer & Anne*

In [5]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

# FPD30 June 2024 Month End

In [6]:
sq = """SELECT t1.new_loan_type,
ROUND(SUM(CASE when t1.defFPD30 = 1 then 1 ELSE 0 end )/sum(t1.obsFPD30)*100,3) as percentage_of_FPD30
from `prj-prod-dataplatform.risk_credit_mis.loan_master_table` t1
inner JOIN prj-prod-dataplatform.risk_credit_mis.loan_bucket_flow_report_core t2
ON
t1.loanAccountNumber = t2.loanAccountNumber
WHERE t1.flagDisbursement = 1
and t1.new_loan_type in ('Quick','Flex','SIL-Instore')
and t2.loanStatus in ('In Arrears','Normal')
and t2.bucketDate in ('2024-06-30')
GROUP BY 1;"""

june24fpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
june24fpd30

Job ID 5893b079-ea58-4540-b884-c309e8b6e1c2 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,new_loan_type,percentage_of_FPD30
0,SIL-Instore,6.831
1,Quick,10.838
2,Flex,1.345


# 30+, 60+, 90+ for June 2024

In [7]:
sq = """with b as
(select t1.new_loan_type 
, count(distinct t2.loanAccountNumber) cnt_loan
, count(distinct case when t2.Max_current_DPD >=30 then t2.loanAccountNumber end) thirtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=30 then t2.loanAccountNumber end) obsthirtyplus
, count(distinct case when t2.Max_current_DPD >=60 then t2.loanAccountNumber end) sixtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=60 then t2.loanAccountNumber end) obssixtyplus
, count(distinct case when t2.Max_current_DPD >=90 then t2.loanAccountNumber end) nintyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >= 90 then t2.loanAccountNumber end) obsnintyplus
from `risk_credit_mis.loan_bucket_flow_report_core` t2
inner join `risk_credit_mis.loan_master_table` t1 on t2.loanAccountNumber = t1.loanAccountNumber
WHERE t1.flagDisbursement = 1
and t1.new_loan_type in ('Quick','Flex','SIL-Instore')
and t2.loanStatus in ('In Arrears','Normal')
and t2.bucketDate in ('2024-06-30')
group by 1
)
select 
new_loan_type,
round(sum(thirtyplus)/sum(obsthirtyplus)*100, 3) thirtyplus,
round(sum(sixtyplus)/sum(obsthirtyplus)*100, 3) sixtyplus,
round(sum(nintyplus)/sum(obsnintyplus)*100, 3) nintyplus,
from b
group by 1
order by 1
;"""

secondpartjune24 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
secondpartjune24

Job ID d579071b-3118-4b3f-8cce-06e6d2df45b6 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,new_loan_type,thirtyplus,sixtyplus,nintyplus
0,Flex,37.320,33.141,28.770
1,Quick,56.998,51.518,53.406
2,SIL-Instore,22.938,17.128,19.962


# 30+, 60+, 90+ for June 2024 (without Product)

In [8]:
sq = """with b as
(select 
count(distinct t2.loanAccountNumber) cnt_loan
, count(distinct case when t2.Max_current_DPD >=30 then t2.loanAccountNumber end) thirtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=30 then t2.loanAccountNumber end) obsthirtyplus
, count(distinct case when t2.Max_current_DPD >=60 then t2.loanAccountNumber end) sixtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=60 then t2.loanAccountNumber end) obssixtyplus
, count(distinct case when t2.Max_current_DPD >=90 then t2.loanAccountNumber end) nintyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >= 90 then t2.loanAccountNumber end) obsnintyplus
from `risk_credit_mis.loan_bucket_flow_report_core` t2
inner join `risk_credit_mis.loan_master_table` t1 on t2.loanAccountNumber = t1.loanAccountNumber
WHERE t1.flagDisbursement = 1
and t1.new_loan_type in ('Quick','Flex','SIL-Instore')
and t2.loanStatus in ('In Arrears','Normal')
and t2.bucketDate in ('2024-06-30')
)
select 
round(sum(thirtyplus)/sum(obsthirtyplus)*100, 3) thirtyplus,
round(sum(sixtyplus)/sum(obsthirtyplus)*100, 3) sixtyplus,
round(sum(nintyplus)/sum(obsnintyplus)*100, 3) nintyplus,
from b
order by 1
;"""

thirdpartjune24 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
thirdpartjune24

Job ID 5e291335-7dc1-45a8-abc7-766ccc3aa06f successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,thirtyplus,sixtyplus,nintyplus
0,29.413,23.738,27.542


In [9]:
dfjune24 = june24fpd30.merge(secondpartjune24, on='new_loan_type', how='left')
dfjune24

,new_loan_type,percentage_of_FPD30,thirtyplus,sixtyplus,nintyplus
0,SIL-Instore,6.831,22.938,17.128,19.962
1,Quick,10.838,56.998,51.518,53.406
2,Flex,1.345,37.320,33.141,28.770


In [10]:
resultjune24 = pd.concat([dfjune24, thirdpartjune24], ignore_index=True)
resultjune24

,new_loan_type,percentage_of_FPD30,thirtyplus,sixtyplus,nintyplus
0,SIL-Instore,6.831,22.938,17.128,19.962
1,Quick,10.838,56.998,51.518,53.406
2,Flex,1.345,37.320,33.141,28.770
3,NaN,NaN,29.413,23.738,27.542


In [11]:
resultjune24['new_loan_type'] = resultjune24['new_loan_type'].fillna("Portfolio")
resultjune24

,new_loan_type,percentage_of_FPD30,thirtyplus,sixtyplus,nintyplus
0,SIL-Instore,6.831,22.938,17.128,19.962
1,Quick,10.838,56.998,51.518,53.406
2,Flex,1.345,37.320,33.141,28.770
3,Portfolio,NaN,29.413,23.738,27.542


In [9]:
import pandas as pd
from openpyxl import load_workbook


In [12]:
# Load the existing Excel file
existing_excel_file = r'C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Collection\Jennifer Zaldivar\KRI_New_Report\Data\202406011_KRI_Report_New.xlsx'
xls = pd.ExcelFile(existing_excel_file)

# Read each existing sheet into separate dataframes
df_january = pd.read_excel(xls, sheet_name='January2024')
df_february = pd.read_excel(xls, sheet_name='February2024')
df_march = pd.read_excel(xls, sheet_name='March2024')
df_april = pd.read_excel(xls, sheet_name='April2024')
df_may = pd.read_excel(xls, sheet_name='May2024')


# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter(r'C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Collection\Jennifer Zaldivar\KRI_New_Report\Data\20240712_KRI_Report_New.xlsx', engine='xlsxwriter')

# Write each dataframe to a separate worksheet
df_january.to_excel(writer, sheet_name='January2024')
df_february.to_excel(writer, sheet_name='February2024')
df_march.to_excel(writer, sheet_name='March2024')
df_april.to_excel(writer, sheet_name='April2024')
df_may.to_excel(writer, sheet_name='May2024')
resultjune24.to_excel(writer, sheet_name='June2024')

# Close the Pandas Excel writer and output the Excel file
writer.close()
